Idea:

Assume only few spoofing attacks

Synchronize clocks assuming all broadcast locations are correct

Estimate positions for messages based on sensor timestamps

Remove messages with high deviations between broadcast locations and estimated locations

Re-synchronize clocks and re-check broadcast locations

In [15]:
import pandas as pd
import numpy as np
import os
import re
import position_estimator
from position_estimator import GeoPoint
from collections import defaultdict
from tqdm.notebook import tqdm
import itertools

DATA_LOCATION = "E:/thesis_data/1hour_complete_22_01_10_05/"
#DATA_LOCATION = "../raw_data/1hour/"

sensors = set()
#sensors_rev = dict()
received = defaultdict(lambda: { "sensors": list() })
sensor_received = defaultdict(list)

MSG_CUTOFF = 100000
#MSG_CUTOFF = 1

pbar = tqdm(total=MSG_CUTOFF, mininterval=1)
pbar.update(0)

discarded = defaultdict(int)
total = 0

for file in os.listdir(DATA_LOCATION):
    assert re.match(r"^part-\d{5}$", file)
    pbar.set_description(file)
    pbar.refresh()
    with open(os.path.join(DATA_LOCATION, file), "r") as f:
        line = f.readline()
        if not line:
            continue
        assert line == "sensorType,sensorLatitude,sensorLongitude,sensorAltitude,timeAtServer,timeAtSensor,timestamp,rawMessage,sensorSerialNumber,RSSIPacket,RSSIPreamble,SNR,confidence\n"
        while (line := f.readline().strip()):
            sensorType,sensorLatitude,sensorLongitude,sensorAltitude,timeAtServer,timeAtSensor,timestamp,rawMessage,sensorSerialNumber,RSSIPacket,RSSIPreamble,SNR,confidence = line.split(',')
            
            total += 1
            if not position_estimator.is_relevant(rawMessage):
                discarded["irrelevant"] += 1
                continue

            if 'null' in [sensorLatitude, sensorLongitude, sensorAltitude, timeAtSensor, timestamp]:
                discarded["null_values"] += 1
                continue
            
            sensorLatitude = float(sensorLatitude)
            sensorLongitude = float(sensorLongitude)
            sensorAltitude = float(sensorAltitude)
            timeAtSensor = float(timeAtSensor)
            timestamp = float(timestamp)

            timeAtServer = float(timeAtServer)

            #print(timestamp, timestamp/2**30)
            assert 0 <= timestamp / 2**30 < 1

            timeAtSensor += timestamp / 2**30

            # limit to europe for now
            if not 35 < sensorLatitude < 75:
                discarded["invalid_sensor_pos_lat"] += 1
                continue
            elif not -10 < sensorLongitude < 40:
                discarded["invalid_sensor_pos_lon"] += 1
                continue

            if not (sensorLatitude and sensorLongitude):
                discarded["invalid_sensor_pos_zero"] += 1
                continue

            if not "pos" in received[rawMessage]:
                try:
                    received[rawMessage]["pos"] = position_estimator.get_announced_pos(rawMessage, timeAtSensor)
                    assert -90 <= received[rawMessage]["pos"].lat <= 90
                    assert -180 <= received[rawMessage]["pos"].lon <= 180
                except:
                    discarded["invalid_msg_pos"] += 1
                    #print("Couldn't get position :(")
                    del received[rawMessage]
                    continue

            sensor = (GeoPoint(sensorLatitude, sensorLongitude, sensorAltitude), sensorType)
            if sensor not in sensors:
                #sensors_rev[sensor] = len(sensors)
                sensors.add(sensor)

            if (timeAtSensor, sensor) not in received[rawMessage]["sensors"]:
                received[rawMessage]["sensors"].append((timeAtSensor, sensor))

            sensor_received[sensor].append((timeAtSensor, timeAtServer))

    pbar.n = len(received)
    pbar.refresh()

    print("total:", total)
    print(discarded)
    print("sum discarded:", sum(discarded.values()))

    if len(received) >= MSG_CUTOFF:
        break # memory constraint


print("Sensors:", len(sensors))
print("Relevant Received Messages:", len(received))


  0%|          | 0/1 [00:00<?, ?it/s]

total: 1367359
defaultdict(<class 'int'>, {'irrelevant': 1046924, 'invalid_sensor_pos_lon': 101457, 'invalid_sensor_pos_lat': 60074, 'null_values': 8097, 'invalid_msg_pos': 100})
sum discarded: 1216652
Sensors: 725
Relevant Received Messages: 15925


In [16]:
for sensor in sensor_received:
    #print(sensor_received[sensor])
    #print(sorted(sensor_received[sensor]))
    break

[(46048.27085663844, 1641790800.0), (46048.272087960504, 1641790800.0), (46048.27114101313, 1641790800.0), (46048.27265653759, 1641790800.001), (46048.27361750323, 1641790800.22), (46048.27498498093, 1641790800.22), (46048.27722696029, 1641790800.437), (46048.27661402803, 1641790800.437), (46048.27613704372, 1641790800.437), (46048.27779738605, 1641790800.655), (46048.2796248924, 1641790800.656), (46048.28016261011, 1641790800.882), (46048.28018236719, 1641790800.882), (46048.28138307668, 1641790800.883), (46048.28219761327, 1641790800.883), (46048.28259443957, 1641790801.091), (46048.28249162529, 1641790801.091), (46048.283709120005, 1641790801.092), (46048.28437454067, 1641790801.092), (46048.28583419323, 1641790801.31), (46048.287447931245, 1641790801.533), (46048.2875620909, 1641790801.533), (46048.2876322018, 1641790801.533), (46048.28952070791, 1641790801.534), (46048.288294277154, 1641790801.534), (46048.28940380085, 1641790801.534), (46048.290573667735, 1641790801.752), (46048.

In [17]:
def del_sensor(sensor):
    assert type(sensor) is tuple
    assert len(sensor) == 2
    assert type(sensor[0]) is GeoPoint
    assert type(sensor[1]) is str
    sensors.remove(sensor)
    del sensor
    

print("Sensors before:", len(sensors))

for sensor, timestamps in sensor_received.items():
    if len(set([e[0] for e in timestamps])) < len(timestamps):
        # non-unique timestamps -> weird sensor behavior
        print(sensor, len(timestamps), len(set([e[0] for e in timestamps])))
        #print(timestamps)
        #print(set([e[0] for e in timestamps]))
        del_sensor(sensor)
        continue
    #for i in range(len(timestamps) - 1):
    #    if timestamps[i][0] > timestamps[i+1][0] + 100:
    #        del_sensor(sensor)
    #        break
            #print(sensor)
            #print(i, timestamps[i], timestamps[i+1])
            #print("td_sensor:", timestamps[i][0] - timestamps[i+1][0], "td_server:", timestamps[i][1] - timestamps[i+1][1])

# clean up received data structure
for msg in received:
    for i in reversed(range(len(received[msg]["sensors"]))):
        if received[msg]['sensors'][i][1] not in sensors:
            del received[msg]['sensors'][i]

            

print("Sensors after:", len(sensors))

Sensors before: 725
(Latitude: 47.506505, Longitude: 8.023311, Altitude: 346.0, 'dump1090') 108 54
(Latitude: 53.34033, Longitude: -6.32304, Altitude: 33.0, 'dump1090') 260 258
(Latitude: 49.569286, Longitude: 6.064, Altitude: 343.0, 'dump1090') 2910 1015
(Latitude: 49.569286, Longitude: 6.064015, Altitude: 343.0, 'dump1090') 2961 933
(Latitude: 49.1641, Longitude: 2.8778, Altitude: 132.0, 'dump1090') 597 395
(Latitude: 48.055249, Longitude: 11.609701, Altitude: 1873.0, 'dump1090') 134 37
(Latitude: 48.21188, Longitude: 11.25848, Altitude: 520.0, 'dump1090') 383 81
(Latitude: 51.9708, Longitude: 7.5958, Altitude: 66.0, 'dump1090') 101 97
Sensors after: 717


In [18]:
for sensor in sensor_received:
    ts = defaultdict(int)
    for t in sensor_received[sensor]:
        ts[t[0]] += 1
    #print(sorted(ts.items()))

In [19]:
# remove messages that have been sent more than once
print("Messages before:", len(received))
for msg in list(received.keys()):
    if len(set([s for t, s in received[msg]["sensors"]])) < len(received[msg]["sensors"]):
        del received[msg]
print("Messages after: ", len(received))

Messages before: 15925
Messages after:  15140


In [20]:
from position_estimator import GeoPoint
import traceback
import ipyparallel as ipp
import util
C = 299792458 # light speed, meters per second

n_cores = 16
n_messages_to_process = 32

time_delta = defaultdict(lambda: defaultdict(list))
to_process = [(e['pos'], e['sensors']) for e in received.values()]#[:n_messages_to_process]
print(to_process[0])
with ipp.Cluster(n=n_cores) as rc:
    view = rc.load_balanced_view()
    asyncresult = view.map_async(util.calc_timedeltas, *zip(*to_process))
    asyncresult.wait_interactive()
    for td in tqdm(asyncresult.get()):
        for s1 in td:
            for s2 in td[s1]:
                time_delta[s1][s2].extend(td[s1][s2])



'''for msg in tqdm(received):
    if "pos" not in received[msg] or received[msg]["pos"] is None:
        continue

    if len(received[msg]["sensors"]) < 2:
        continue

    #for t, s in list(received[msg]["sensors"]):
    #    dists = sorted([s[0].dist(e[1][0]) for e in received[msg]["sensors"] if e[1] != s])
    #    if dists[int(len(dists)/10)] > 1e6: # ge 1000 km
    #        print("removing:", t, s, dists)
    #        received[msg]["sensors"].remove((t, s))

    try:
        sensor_dists_to_msg_origin = {x[1]: received[msg]["pos"].dist(x[1][0]) for x in received[msg]["sensors"]}
        time_to_sensor = { s: x / C for s, x in sensor_dists_to_msg_origin.items() }
    except:
        traceback.print_exc()
        print(received[msg]["pos"].lat, received[msg]["pos"].lon, received[msg]["pos"].alt)
        print({x[1]: (x[1][0].lat, x[1][0].lon, x[1][0].alt) for x in received[msg]["sensors"]})

    for (t1, s1), (t2, s2) in itertools.combinations(received[msg]["sensors"], 2):
        assert s1 != s2
        td = (t1 - time_to_sensor[s1]) - (t2 - time_to_sensor[s2])
        time_delta[s1][s2].append(td)
        time_delta[s2][s1].append(-td)
'''


lens = []
for s1 in time_delta:
    for s2 in time_delta[s1]:
        lens.append(len(time_delta[s1][s2]))

np.histogram(lens)

(Latitude: 52.56525, Longitude: 4.23868, Altitude: 2331.7200000000003, [(46048.27085663844, (Latitude: 52.356074, Longitude: 4.63139, Altitude: 9.0, 'dump1090')), (50744.40814461373, (Latitude: 52.2108, Longitude: 4.4042, Altitude: 39.0, 'dump1090'))])
Starting 16 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/16 [00:00<?, ?engine/s]

calc_timedeltas:   0%|          | 0/15140 [00:00<?, ?tasks/s]

  0%|          | 0/15140 [00:00<?, ?it/s]

Stopping engine(s): 1645281205
engine set stopped 1645281205: {'engines': {'0': {'exit_code': 1, 'pid': 29148, 'identifier': '0'}, '1': {'exit_code': 1, 'pid': 34576, 'identifier': '1'}, '2': {'exit_code': 1, 'pid': 33476, 'identifier': '2'}, '3': {'exit_code': 1, 'pid': 1236, 'identifier': '3'}, '4': {'exit_code': 1, 'pid': 32660, 'identifier': '4'}, '5': {'exit_code': 1, 'pid': 24160, 'identifier': '5'}, '6': {'exit_code': 1, 'pid': 35040, 'identifier': '6'}, '7': {'exit_code': 1, 'pid': 34272, 'identifier': '7'}, '8': {'exit_code': 1, 'pid': 24108, 'identifier': '8'}, '9': {'exit_code': 1, 'pid': 34572, 'identifier': '9'}, '10': {'exit_code': 1, 'pid': 21412, 'identifier': '10'}, '11': {'exit_code': 1, 'pid': 35004, 'identifier': '11'}, '12': {'exit_code': 1, 'pid': 33640, 'identifier': '12'}, '13': {'exit_code': 1, 'pid': 32508, 'identifier': '13'}, '14': {'exit_code': 1, 'pid': 22288, 'identifier': '14'}, '15': {'exit_code': 1, 'pid': 34888, 'identifier': '15'}}, 'exit_code': 1}
S

(array([54000,  9058,  3080,  1366,   582,   238,    62,    18,     4,
            2], dtype=int64),
 array([  1. ,  67.6, 134.2, 200.8, 267.4, 334. , 400.6, 467.2, 533.8,
        600.4, 667. ]))

In [5]:
#import pickle
#
#
#with open("temp_storage/received.pickle", "wb") as f:
#    pickle.dump(dict(received), f)
#
#with open("temp_storage/sensors.pickle", "wb") as f:
#    pickle.dump(sensors, f)
#
#with open("temp_storage/time_delta.pickle", "wb") as f:
#    pickle.dump(time_delta, f)

PicklingError: Can't pickle <function <lambda> at 0x000001788634D310>: attribute lookup <lambda> on __main__ failed

We now have all time deltas between stations that picked up the same signal.
We model the clock shifts using gaussian distributions, with some mean mu and standard deviation sigma

Then, to get time delta probability distribution between two stations that didn't measure the same message, we convolute the probability density functions of stations with directly known time-delta

Example:
sensors A and B measured the same 100 messages, so we can directly model their time delta distribution function as: td_AB = Gauss(mu_AB, sigma_AB)
Similarly, assume we estimate the density function for sensors B and C as Gauss(mu_BC, sigma_BC)

Now, to get the density function for sensors A and C, we can convolute these two density functions:
density function of time delta A-C = Gauss(mu_AB, sigma_AB) * Gauss(mu_BC, sigma_BC) = Gauss(mu_AB + mu_BC, sqrt(sigma_AB^2 + sigma_BC^2))

We do this for all paths going from A to C, and sum up all PDFs to get the final PDF for the time delta between A and C.
For efficiency, we could just do a "shortest path", where the path lengths are the sum of variances.
This way, we just take the "best" path, with least uncertainty.

In [21]:
import statistics

time_delta_gaussians =  defaultdict(lambda: defaultdict(lambda: None))

# initialize with directly known time deltas
for i in tqdm(time_delta.keys()):
    for j in time_delta[i].keys():
        if len(time_delta[i][j]) > 1:
            mean = statistics.mean(time_delta[i][j])
            var = statistics.variance(time_delta[i][j], xbar=mean) / len(time_delta[i][j])
            if var <= 1e-9 or True:
                time_delta_gaussians[i][j] = (mean, var)

#print(time_delta)
#print(sorted([statistics.mean([e[1] for e in time_delta_gaussians[a].values()]) for a in time_delta_gaussians]))
#%exit
# floyd's algorithm
for k in tqdm(time_delta.keys()):
    for i in time_delta.keys():
        for j in time_delta.keys():
            if i == j or j == k or k == i:
                continue
            if k not in time_delta[i] or j not in time_delta[k]:
                continue
            if time_delta_gaussians[i][k] is not None and time_delta_gaussians[k][j] is not None:
                new_variance = time_delta_gaussians[i][k][1] + time_delta_gaussians[k][j][1]
                if time_delta_gaussians[i][j] is None or time_delta_gaussians[i][j][1] > new_variance:
                    new_mean = time_delta_gaussians[i][k][0] + time_delta_gaussians[k][j][0]
                    time_delta_gaussians[i][j] = (new_mean, new_variance)


  0%|          | 0/706 [00:00<?, ?it/s]

  0%|          | 0/706 [00:00<?, ?it/s]

In [22]:
variances = [ time_delta_gaussians[i][j][1] for i,j in itertools.combinations(time_delta.keys(), 2) if time_delta_gaussians[i][j] is not None ]
num_conns = [ len(time_delta[i][j]) for i,j in itertools.combinations(time_delta.keys(), 2) if len(time_delta[i][j]) > 0]

print("Mean Variance:", statistics.mean(variances))
print("Median variance:", statistics.median(variances))
print("Variance modes:", statistics.multimode(variances))

print("Mean nConnections:", statistics.mean(num_conns))
print("Median nConnections:", statistics.median(num_conns))
print("nConnections modes:", statistics.multimode(num_conns))

Mean Variance: 517656.56668340805
Median variance: 2.0870260435758523e-11
Variance modes: [1098186886.358807]
Mean nConnections: 43.96120450226575
Median nConnections: 21
nConnections modes: [1]


Using those time deltas, we can now check the aircraft locations

In [32]:
import pyproj
import warnings
import math
warnings.filterwarnings("ignore")



def to_ecef(geopoint):
    ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
    lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')
    x, y, z = pyproj.transform(lla, ecef, geopoint.lon, geopoint.lat, geopoint.alt, radians=False)
    return (x, y, z)

def to_wgs84(x, y, z):
    ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
    lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')
    lon, lat, alt = pyproj.transform(ecef, lla, x, y, z, radians=False)
    return (lat, lon, alt)

def lorentz_inner(u, v):
    assert len(u) == len(v) == 4
    return u[0]*v[0] + u[1]*v[1] + u[2]*v[2] - u[3]*v[3]


iterations = 0

for msg, data in tqdm(received.items()):
    #print(time_delta_gaussians)
    #break

    if len(data["sensors"]) < 4:
        continue

    iterations += 1
    if iterations >= 5:
        break

    #valid_sensors = [ data["sensors"][0] ]
    #for s in data["sensors"][1:]:
    #    if s[]

    # Bancroft method
    B = list()
    a = list()
    td_base = None


    best_variances = list()
    for t1, s1 in data["sensors"]:
        var_sum = 0
        non_none = 0
        for t2, s2 in data["sensors"]:
            if s1 == s2:
                continue
            if s2 in time_delta_gaussians[s1]:
                non_none += 1
                var_sum += time_delta_gaussians[s1][s2][1]
        best_variances.append((-non_none, var_sum, t1, s1))

    best_variances.sort()

    if best_variances[0][0] > -4:
        # less than four connected sensors
        continue

    for _, _, timestamp, sensor in best_variances:
        ecef_coords = to_ecef(sensor[0])
        if td_base is None:
            td_base = sensor
            td = 0
        else:
            if sensor in time_delta_gaussians[td_base] and time_delta_gaussians[td_base][sensor] is not None:
                td = time_delta_gaussians[td_base][sensor][0]
            
        print(timestamp, td, td*C)
        B.append([*ecef_coords, C*(-timestamp - td)])
        a.append(0.5*(ecef_coords[0]**2 + ecef_coords[1]**2 + ecef_coords[2]**2 - td**2))
        if len(a) >= 4:
            break


    e = [1] * 4#len(data["sensors"])

    print(B)
    B_plus = np.matmul(np.linalg.inv(np.matmul(np.transpose(B), B)), np.transpose(B))
    lambda_coefs = (lorentz_inner(np.matmul(B_plus,e), np.matmul(B_plus,e)), 2*(lorentz_inner(np.matmul(B_plus, a), np.matmul(B_plus,e)) - 1), lorentz_inner(np.matmul(B_plus,a), np.matmul(B_plus,a)))
    lambda_roots = np.roots(lambda_coefs)

    print(lambda_coefs)
    print(lambda_roots)
    #if len(lambda_roots) != 2:
    #    continue
    assert len(lambda_roots) == 2

    best_dist = 1e9
    for root in lambda_roots:
        sol = np.matmul(B_plus, (a + root * np.array(e)))
        print(sol)
        dist = GeoPoint(*to_wgs84(*sol[:3])).dist(data["pos"])
        best_dist = min(best_dist, dist)

    print("Distance:", best_dist)
    

    


  0%|          | 0/15140 [00:00<?, ?it/s]

30173.3061043052 0 0
11902.181495956145 18271.124083472525 5477545199407.226
16714.46839441452 13458.83721536665 4034857890616.643
16198.979887126945 13974.326311133129 4189397633708.6733
[[3804147.8912737486, 399262.1440125886, 5086788.213380139, -9045729602996.06], [3936236.774812599, 328023.7649173559, 4991123.138808308, -9045729445642.035], [3910490.7944052764, 460363.80869302875, 5000884.9796087975, -9045729454741.486], [3891552.5361006926, 318419.72813393536, 5026434.255554042, -9045729631163.023]]
(1.735695831022687e-21, -2.0000404627709507, 236288435278.25494)
[1.15229894e+21 1.18141827e+11]
[-2.90680029e+10 -2.83596134e+09 -3.81001660e+10 -1.27419733e+08]
[ 3.10797393e+05  3.30279251e+04  3.72287244e+05 -1.91052271e+00]
Distance: 5887614.343308
48817.653345110826 0 0
109647.76234902535 -60830.10870305446 -18236407808495.887
121.0970687288791 48696.5559174788 14598860194635.414
106903.0541003095 -58085.40069984268 -17413565049720.756
[[4309287.367993003, 567971.1819096402, 4652

In [11]:
import scipy.optimize

def estimate_position(sensors, time_delta_gaussians):
    assert len(sensors) >= 4

    # we want to find a position p, for which we want to minimize some objective function.
    # this objective function is subject to the time deltas,
    # and the distances from the estimated position and the sensor positions
    # also, we should consider the variances in the time delta distributions:
    # if some time delta probability distribution has very big variance,
    # we can't trust that sensor to the same degree as a sensor with a small time delta variance

    # attempt 1:
    def residual_error(p):
        p = position_estimator.GeoPoint(*p)
        #print("p:", p.lat, p.lon, p.alt)
        err = 0
        for i,j in itertools.combinations(range(len(sensors)), 2):
            if time_delta_gaussians[sensors[i]][sensors[j]] is None:
                continue

            dist_i = p.dist(sensors[sensors[i]][0])
            dist_j = p.dist(sensors[sensors[j]][0])
            t_i = sensors[i][0] - dist_i / C
            t_j = sensors[j][0] - dist_j / C
            #print(sensors[i], sensors[j], t_i, t_j, time_delta_gaussians[sensors[i]][sensors[j]])
            e = (t_i - t_j - time_delta_gaussians[sensors[i]][sensors[j]][0]) ** 2 / time_delta_gaussians[sensors[i]][sensors[j]][1]
            err += e
        #print(err)
        return err

    def residual_error2(p):
        p = position_estimator.GeoPoint(*p)
        err = 0
        true_t = sensor_timestamps[0] - p.dist(sensors[0][1][0]) / C
        for i in range(1, len(sensors)):
            t = sensors[i][0] - p.dist(sensors[sensors[i]][0]) / C
            err += true_t - t - time_delta_gaussians[sensors[0]][sensors[i]][0]
        #print(err, len(sensors), err/len(sensors))
        return err

    def residual_error3(p):
        p = position_estimator.GeoPoint(*p)
        err = 0
        for assumed_true in range(len(sensors)):
            true_t = sensor_timestamps[sensors[assumed_true]] - p.dist(sensors[sensors[assumed_true]][0]) / C
            for i in range(len(sensors)):
                if i == assumed_true:
                    continue
                t = sensors[i][0] - p.dist(sensors[sensors[i]][0]) / C
                err += true_t - t - time_delta_gaussians[sensors[assumed_true]][sensors[i]][0]
            #print(err, len(sensors), err/len(sensors))
            return err

    def residual_error4(p):
        p = position_estimator.GeoPoint(*p)
        # choose 4 best time_delta distributions (smallest variances)
        val = (100000, None)
        for a, b, c, d in itertools.combinations(range(len(sensors)), 4):
            s = sum([time_delta_gaussians[sensors[e]][sensors[f]][1] for e,f in itertools.combinations([a,b,c,d],2)])
            val = min(val, (s, (a,b,c,d)))
        err = 0
        for i,j in itertools.combinations(val[1], 2):
            #print(i,j, sensors[i], sensors[j])
            ti = sensors[i][0] - p.dist(sensors[sensors[i]][0]) / C
            tj = sensors[j][0] - p.dist(sensors[sensors[j]][0]) / C
            err += ti - tj - time_delta_gaussians[sensors[i]][sensors[j]][0]
        return err


    bounds = scipy.optimize.Bounds([-90, -180, 0], [90, 180, 100000])
    method = 'L-BFGS-B'
    optimum = scipy.optimize.minimize(residual_error, [sensors[0][1][0].lat, sensors[0][1][0].lon, sensors[0][1][0].alt], method=method, bounds=bounds).x
    print("optimum:", optimum)
    return GeoPoint(*optimum)


it = 0
for msg in tqdm(received):
    if len(received[msg]["sensors"]) < 4:
        continue
    
    estimated_pos = estimate_position(received[msg]["sensors"], time_delta_gaussians)
    received[msg]["estimated_pos"] = estimated_pos
    print("Broadcast pos:", received[msg]["pos"].lat, received[msg]["pos"].lon, received[msg]["pos"].alt)
    print("Estimated pos:", received[msg]["estimated_pos"].lat, received[msg]["estimated_pos"].lon, received[msg]["estimated_pos"].alt)
    print("Dist:", received[msg]["pos"].dist(received[msg]["estimated_pos"]))

    it += 1
    if it >= 10:
        break
    

  0%|          | 0/14297 [00:00<?, ?it/s]

optimum: [ 52.14    10.4601 120.    ]
Broadcast pos: 52.67409 9.99306 10972.800000000001
Estimated pos: 52.14 10.4601 120.0
Dist: 68263.95609740308
optimum: [  46.8914239    7.499006  1713.       ]
Broadcast pos: 49.28895 7.45827 10363.2
Estimated pos: 46.8914239 7.4990060000000085 1713.0
Dist: 266743.4218124952
optimum: [51.6303442  4.9487092 45.       ]
Broadcast pos: 51.87618 7.56424 12192.0
Estimated pos: 51.6303442 4.9487092 45.0
Dist: 183067.81057760966
optimum: [ 50.03496   8.24124 312.     ]
Broadcast pos: 49.32985 7.43845 10972.800000000001
Estimated pos: 50.03496 8.24124 312.0
Dist: 98082.82992399957
optimum: [ 51.43095016   6.92255878 119.56099701]
Broadcast pos: 51.43611 7.27287 3268.98
Estimated pos: 51.43095016479492 6.922558784484863 119.56099700927734
Dist: 24570.580713169555
optimum: [51.9899  4.3754 30.    ]
Broadcast pos: 51.9912 -1.62483 11887.2
Estimated pos: 51.9899 4.3754 30.0
Dist: 412224.2846733337
optimum: [51.282448  6.555248 85.      ]
Broadcast pos: 49.1738